## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-17-02-08-18 +0000,bbc,Putin-Trump call a curveball for Zelensky ahea...,https://www.bbc.com/news/articles/cze63r34213o...
1,2025-10-17-02-07-13 +0000,nypost,US carries out another strike on suspected dru...,https://nypost.com/2025/10/16/world-news/us-ca...
2,2025-10-17-02-02-56 +0000,48hills,No Kings protests this weekend: A guide,https://48hills.org/2025/10/no-kings-protests-...
3,2025-10-17-02-01-39 +0000,missionlocal,Supervisor Chyanne Chen tours S.F. ICE facilit...,https://missionlocal.org/2025/10/supervisor-ch...
4,2025-10-17-01-56-43 +0000,nypost,GOP Rep. Dave Taylor claims swastika flag seen...,https://nypost.com/2025/10/16/us-news/gop-rep-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2364/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
1,trump,63
410,new,23
250,will,18
88,china,15
351,he,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
118,2025-10-16-20-59-00 +0000,wsj,The four-star U.S. Navy admiral in charge of U...,https://www.wsj.com/politics/national-security...,140
184,2025-10-16-17-33-15 +0000,nypost,Trump says he will meet with Putin in Budapest...,https://nypost.com/2025/10/16/us-news/trump-sa...,125
244,2025-10-16-14-49-58 +0000,nypost,Trump says CBS under owner David Ellison will ...,https://nypost.com/2025/10/16/media/trump-says...,125
62,2025-10-16-23-01-00 +0000,wsj,Trump Says He Will Meet With Putin in Budapest...,https://www.wsj.com/world/russia/trump-and-put...,124
251,2025-10-16-14-19-27 +0000,bbc,India casts doubt on Trump's claim Modi will s...,https://www.bbc.com/news/articles/c8eykdy0567o...,122


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
118,140,2025-10-16-20-59-00 +0000,wsj,The four-star U.S. Navy admiral in charge of U...,https://www.wsj.com/politics/national-security...
44,68,2025-10-16-23-33-02 +0000,nypost,Zohran Mamdani under fire after NYC mayoral de...,https://nypost.com/2025/10/16/us-news/zohran-m...
346,58,2025-10-16-09-00-00 +0000,wsj,What Happens When Oil and Gas Wells Die? West ...,https://www.wsj.com/business/energy-oil/what-h...
294,47,2025-10-16-11-55-28 +0000,bbc,'Whose side are you on?' MP attacks government...,https://www.bbc.com/news/videos/cly1vj1z2j9o?a...
150,44,2025-10-16-19-13-22 +0000,bbc,At least three dead after Kenya police open fi...,https://www.bbc.com/news/articles/cwyk1l65nqvo...
184,44,2025-10-16-17-33-15 +0000,nypost,Trump says he will meet with Putin in Budapest...,https://nypost.com/2025/10/16/us-news/trump-sa...
71,40,2025-10-16-22-31-31 +0000,startribune,"Prison for woman who admitted she was driving,...",https://www.startribune.com/prison-for-woman-a...
126,35,2025-10-16-20-34-59 +0000,nypost,Ex-Trump national security adviser John Bolton...,https://nypost.com/2025/10/16/us-news/ex-trump...
215,32,2025-10-16-16-00-00 +0000,wsj,An extended government shutdown raises the pro...,https://www.wsj.com/economy/central-banking/fe...
39,31,2025-10-16-23-55-28 +0000,latimes,Mpox strain that may be more severe and easily...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
